In [1]:
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.agent.react import REACT_INSTRUCTION_TABMWP
from agential.cog.prompts.benchmark.tabmwp import TABMWP_FEWSHOT_EXAMPLES_REACT

from langchain_openai import ChatOpenAI

import warnings
warnings.filterwarnings('ignore')

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

In [2]:
question = """Read the following table regarding "Bowling Scores" and then write Python code to answer a question:

Name | Score
Amanda | 117
Sam | 236
Irma | 144
Mike | 164

Question: Some friends went bowling and kept track of their scores. How many more points did Mike score than Irma?"""

agent = ReActAgent(llm=llm, mode={"math": "tabmwp"})

out = agent.generate(
    question=question,
    examples=SVAMP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_SVAMP,
)

<===================================================================>
Answer a math question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be two types:
(1) Calculate[code], which implements code to answer the math question, saving the answer as the `answer` variable.
(2) Finish[code], which returns the code to answer the math question and finishes the task, saving the answer as the `answer` variable.
You have a maximum of 6 steps.

Here are some examples:
Question: James bought 93 red and 10 blue stickers, he used 31 red stickers on his fridge and 7 blue stickers on his laptop. How many red stickers does James have?
Thought 1: First, I need to find out how many red stickers James has left after using some on his fridge.
Action 1: Calculate[
```python
original_red_stickers = 93
used_red_stickers = 31
answer = original_red_stickers + used_red_stickers
```
]
Observation 1: 
```python
origin

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'I need to determine the size of each group of bananas after they are organized.\nAction: Calculate[\n```python\ntotal_bananas = 290\nnum_banana_groups = 2\nanswer = total_bananas / num_banana_groups\n```\n]\nObservation: \n```python\ntotal_bananas = 290\nnum_banana_groups = 2\nanswer = total_bananas / num_banana_groups\n```\nExecution Status: Done\nOutput: answer = 145\nThought: Each group of bananas consists of 145 bananas.\nAction: Finish[\n```python\nanswer = 145\n```\n]\nObservation: \n```python\nanswer = 145\n```'
<===================================================================>
Answer a math question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be two types:
(1) Calculate[code], which implements code to answer the math question, saving the answer as the `answer` variable.
(2) Finish[code], which returns the code to answer the math question and finishes the task, saving the ans